In [2]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
from pathlib import Path
import re
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.ml_utils import Classifier

In [16]:
import numpy as np
a = 0.1**10
# (np.exp(1)**a - 1) / a
np.exp(1)**a -1, a

(1.000000082740371e-10, 1.0000000000000006e-10)

In [3]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET]

train_feature_paths = glob('../feature/raw_main/*_train.gz')
feature_names = [re.search(r'/([^/.]*)_train.gz', path).group(1) for path in train_feature_paths]

train_base = read_pkl_gzip('../input/base_train.gz')
Y = train_base[COLUMN_TARGET].values
train_base.drop(COLUMN_TARGET, axis=1, inplace=True)

In [ ]:
length = int(len(train_feature)*0.8)
params = {}
metric = 'auc'
model_type = 'lgb'
score_list = []

for feature in tqdm(feature_names):
    if feature in COLUMNS_IGNORE:
        continue
        
    train_feature_path = f'../feature/raw_main/{feature}_train.gz'
    train_feature = read_pkl_gzip(train_feature_path)
    
    x_train = pd.DataFrame(train_feature[:length], columns=[feature])
    y_train = Y[:length]
    x_valid = pd.DataFrame(train_feature[length:], columns=[feature])
    y_valid = Y[length:]

    try:
        score, oof_pred, test_pred, feim, _ = Classifier(
            model_type=model_type,
            x_train=x_train,
            y_train=y_train,
            x_valid=x_valid,
            y_valid=y_valid,
            x_test=[],
            params=params,
        )
    except ValueError:
        continue
    score_list.append([feature, score])

In [33]:
score_result = pd.DataFrame(score_list, columns=['feature', 'score'])
score_result.sort_values(by='score', ascending=False, inplace=True)

In [70]:
score_result['score_round'] = score_result['score'].round(2)
score_cnt = score_result['score_round'].value_counts()
score_cnt

0.68    61
0.69    47
0.53    35
0.67    34
0.70    26
0.52    25
0.51    18
0.54    18
0.50    14
0.65    13
0.58    12
0.64    11
0.57    10
0.66     9
0.59     9
0.62     8
0.63     7
0.56     7
0.61     5
0.55     3
0.60     3
0.72     1
0.71     1
Name: score_round, dtype: int64

In [74]:
score_result.to_csv('../eda/103__eda_single_feature_train_AUC_holdout8-2.csv', index=False)

In [87]:
from kaggle_utils import move_feature
raw_score_features = score_result[score_result['score']<0.55]['feature'].values
move_feature(raw_score_features, 'raw_main', 'trush')

/home/yryrgogo/github/ieee-fraud/feature/raw_main/V68_train.gz


SystemExit: 

/home/yryrgogo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
train_paths = glob('../feature/raw_main/*_train.gz')
test_paths = glob('../feature/raw_main/*_test.gz')

train_paths = [path for path in train_paths if path.count('DT') or path.count('day_no')]
test_paths  = [path for path in test_paths  if path.count('DT') or path.count('day_no')]

train_df = parallel_load_data(train_paths)
test_df = parallel_load_data(test_paths)
print(train_df.shape)
print(test_df.shape)

train_df['is_train'] = True
test_df['is_train']  = False
result = pd.concat([train_df, test_df], axis=0)

DIR_INPUT = Path('../input')
file_name = 'ieee__result_single_feature_AUC_055-Over.csv'
file_path = DIR_INPUT / file_name

# result.to_csv(file_path, index=False)

(590540, 2)
(506691, 2)


In [106]:
from gcp_utils import gcs_upload

gcs_upload(blob_name=file_name, file_path=file_path)